# BatSimuPOP

# Simulation of population changes on bat colonies over time to assess the use of temporal sampling

# Input file:
1 Genepop file with 2 subpopulations (total number of individuals in genepop is inferior to the number of individuals in the simulation)

subpop 1: local colony that will undergo changes (additional info on age, age category and sex).

subpop 2: Rest of the population. Present to maintain gene flow over time. (additional info on sex and whether juvenile or adult)

Information on sex, age in years (for 1 subpop) or age category (juvenile-adult) is also available (file in fstat format 14 microsatellite loci + additional info)

In [22]:
from simuOpt import setOptions
setOptions(quiet=True)
import simuPOP as sim
from simuPOP.utils import importPopulation

pop = importPopulation('GENEPOP', 'Genpop_2subpops.txt')


Ther are 68 and 192 bats in two populations. No sex and age information yet.

In [23]:
# get age, agecat, sex from fstat file
%preview  pop_dump.txt -n --limit 500

pop.addInfoFields(['age', 'age_cat'])
with open('Fstat_dat(age,agecat,sex).txt') as fs:
    for idx, line in enumerate(fs):
        if idx < 10:
            # skip the first few lines
            continue
        if line.startswith('1 '):
            age, age_cat, sex = line.strip().split()[-3:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            pop.individual(idx-18).age = int(age)
            pop.individual(idx-18).age_cat = int(age_cat)
        elif line.startswith('2 '):
            age_cat, sex = line.strip().split()[-2:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            # no age info
            pop.individual(idx-18).age = 5
            pop.individual(idx-18).age_cat = int(age_cat)
sim.dump(pop, width=3, max=300, output='pop_dump.txt')            

> pop_dump.txt (26.8 KiB):

Ploidy: 2 (diploid)
Chromosomes:
1:  (AUTOSOME, 14 loci)
  A24 (1), B22 (2), CloneA2 (3), Kpa24_2 (4), Mluc8 (5),
  Kpa16_2 (6), CA38 (7), CA11 (8), Mnatt6 (9), G6 (10),
  C112_2 (11), MS3D02 (12), H23_Mluc (13), B23 (14)
Information fields: 
age age_cat 
population size: 260 (2 subpopulations with 68, 192 Individuals)
Number of ancestral populations: 0

SubPopulation 0 (), 68 Individuals:
   0: FU   2  5  1  7  2  4  5  8  5  2  3  7  8  7 |   4  5  3  9  3  9 11 11  5  4  6 14 14 11 |  2 0
   1: FU   8  5  1 11  3  4  7  4  3  1  3 14  8  6 |   9  5  2 14  5  8 10 10  7  1  3 16 10  7 |  2 0
   2: FU   4  6  1  6  3  4  1 10  5  2  3 11  9  6 |   8  6  3  7  6 13  9 10 10  3  4 16 12 13 |  2 0
   3: FU   8  5  1 14  3  6  5  5  4  1  3  7  9  4 |   8  6  3 15  3  8  9 10  4  4  9  7 12 11 |  3 0
   4: FU   6  5  1  4  1  7  9 10  4  1  3  7  8  4 |   8  6  5  6  5  9 12 10  5  4  6 10 10  7 |  3 0
   5: FU   7  5  3 11  4  7  1 10  5  1  3  7  9  7 |   8  6  3 12 10 12  3 10  7  1  4

# Life cycle of species to consider for overlapping generations:
Maxlifespan ~ 17 years old

minMatingAge ~ 2 years old

maxMatingAge ~ 17 years old

Number of young per year ~ 1

1 generation ~ 5 years

# Other settings:

Random mating in subpopulations

Maintain gene flow between both sub-populations using Migrator. This is sex-biased as females remain in the same colony and males disperse.

Female migration rate	0.001

Male migration rate	0.1


# Example scenario:

Simulation of 20 generations (~200 years)

The initial population size is based on Ne estimates from the Genepop file (N genotypes < Actual population size), for example:

N0subpop1=130

N0subpop2=550

Subpop 2 remains constant over time

Subpop 1 undergoes an instant population change (InstantChangeModel) at generation 5 and declines to 80.

Then at generation 12 it undergoes a linear growth at rate 0.2 to a population of 150 (LinearGrowthModel) => Need of MultiStageModel 

# Outputs:

Need Genepop file outputs for each generation (or every 5 years) after breeding to include juveniles. The effective population size estimates will be done separately.

Output info on age and sex of individuals would also help with Ne estimates.